In [85]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import ttest_ind, levene
from sklearn.linear_model import LassoCV
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns #
import matplotlib.pyplot as plt #
from sklearn.metrics import roc_curve, roc_auc_score, classification_report #
from sklearn.metrics import accuracy_score, classification_report

In [86]:
train_df = pd.read_csv(r"D:\wxx\SFYdataJPG\Code\imagescore5\imagescore5\train_normalize.csv")
test_df = pd.read_csv(r"D:\wxx\SFYdataJPG\Code\imagescore5\imagescore5\test_normalize.csv")
# 计算训练集和测试集的特征均值和标准差，检查是否存在显著性差异
assert train_df.shape[1]==test_df.shape[1], "训练集和测试集的特征数量应保持一致！"
# type(train_df.shape)#元组类型
nFeatures = train_df.shape[1]-2  #16个特征
featureName = train_df.columns[2:]#获取列名

In [87]:
nFeatures
featureName

Index(['age', 'gestational_week', 'LA_diameter', 'RA_diameter', 'LV_diameter',
       'RV_diameter', 'LA_area', 'LV_area', 'RA_area', 'RV_area',
       'LA_RA_area_ratio', 'LV_RV_area_ratio',
       'interventricular_septal_thickness', ' LV_ellipticity',
       ' RV_ellipticity', 'Image_Score'],
      dtype='object')

In [88]:
# selected_columns = [ 'age','gestational_week',	'LA_diameter','RA_diameter',
#                     'LV_diameter','RV_diameter','LA_area','LV_area',
#                     'RV_area','RA_area','LA_RA_area_ratio','LV_RV_area_ratio','interventricular_septal_thickness',' LV_ellipticity',' RV_ellipticity']#多
selected_columns = [ 'gestational_week',	'LA_diameter',
                    'LV_diameter','LA_area','LV_area',
                    'RV_area','LA_RA_area_ratio','LV_RV_area_ratio',' LV_ellipticity']#单
# selected_columns = [ 'LV_diameter','LA_RA_area_ratio','LV_RV_area_ratio']#多

In [89]:
X = train_df[selected_columns]
Y = train_df.iloc[:,1]
print(X)
# print(Y) #y是label,x是特征

     gestational_week  LA_diameter  LV_diameter   LA_area   LV_area   RV_area  \
0            1.561094     1.352102     1.569796  2.510497  1.408111  1.053000   
1           -0.327506    -0.103360    -0.216287  0.286516 -0.410530 -0.476191   
2            0.855463     0.912079     1.051256  0.223547  1.003429  0.650061   
3            0.855463     1.081318     1.339334  0.174358 -0.263863  0.332424   
4            0.896971     1.589038     0.964833  2.166870  1.444373  0.500442   
..                ...          ...          ...       ...       ...       ...   
159         -0.763336    -1.761910    -1.195752 -1.116772 -1.052601 -0.861644   
160         -2.672690    -2.195164    -2.022536 -1.373533 -1.306033 -1.394740   
161         -2.278367    -1.504665    -2.146409 -1.155321 -1.250763  0.028015   
162         -0.701075    -0.712623    -1.455022 -0.945308 -1.102344  0.963290   
163         -0.369013    -0.949559    -1.051713  1.101104 -0.518778  2.613211   

     LA_RA_area_ratio  LV_R

In [90]:
model_rf = RandomForestClassifier(n_estimators = 200
                                  , criterion = 'entropy'
                                  , random_state = 20
                                  , class_weight = 'balanced'
                                 )
model_rf.fit(X, Y)
X_test=test_df[selected_columns]
y_test = test_df.iloc[:,1]
# print(len(X_test))
print(model_rf.score(X_test,y_test))

0.9047619047619048


In [91]:
model_dt = DecisionTreeClassifier(criterion='entropy',  # 使用信息熵作为分裂标准
                                  random_state=20,     # 确保结果可重复
                                  class_weight='balanced')  # 处理类别不平衡
model_dt.fit(X, Y)
X_test=test_df[selected_columns]
y_test = test_df.iloc[:,1]
# print(len(X_test))
print(model_dt.score(X_test,y_test))

0.9047619047619048


In [92]:
from sklearn.svm import SVC

# 训练模型
model_svm = SVC(probability=True,  # 启用概率估计
                random_state=20,   # 确保结果可重复
                class_weight='balanced')  # 处理类别不平衡
model_svm.fit(X, Y)
X_test=test_df[selected_columns]
y_test = test_df.iloc[:,1]
# print(len(X_test))
print(model_svm.score(X_test,y_test))

0.9047619047619048


In [93]:
from xgboost import XGBClassifier

# 训练模型
model_xgb = XGBClassifier(n_estimators=200,      # 树的数量
                          objective='binary:logistic',  # 二分类任务
                          random_state=20,       # 确保结果可重复
                          scale_pos_weight=len(Y[Y == 0]) / len(Y[Y == 1]))  # 处理类别不平衡
model_xgb.fit(X, Y)
X_test=test_df[selected_columns]
y_test = test_df.iloc[:,1]
# print(len(X_test))
print(model_xgb.score(X_test,y_test))

0.9047619047619048


In [75]:
# 预测标签
# pred_labels = model_rf.predict(X_test)

# 预测概率
pred_probs = model_xgb.predict_proba(X_test)
print(pred_probs)
# 创建一个 DataFrame 保存结果
results_df = pd.DataFrame({
    'nname': test_df.iloc[:,0],
    'label': y_test,
    # 'Predicted_Label': pred_labels,
    'pred_probability': pred_probs[:,1],  # 类别1的概率
    
    # 如果有更多类别，请继续添加
})

# 导出结果到 CSV 文件
results_df.to_csv(r'D:\wxx\SFYdataJPG\Code\imagescore5\imagescore5\ML\0728-3feaure\tc_xgb.csv', index=False)

print("Results saved to prediction_results.csv")

[[0.99573207 0.00426793]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.99254423 0.00745578]
 [0.7091618  0.29083818]
 [0.9962221  0.00377794]
 [0.99573207 0.00426793]
 [0.9962221  0.00377794]
 [0.99347913 0.00652089]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.99573207 0.00426793]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.99573207 0.00426793]
 [0.01449394 0.98550606]
 [0.9908049  0.00919507]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.9962221  0.00377794]
 [0.00390053 0.9960995 ]
 [0.00390053 0.9960995 ]
 [0.00390053 0.9960995 ]
 [0.00390053 0.9960995 ]
 [0.00390053 0.9960995 ]
 [0.95625126 0.04374875]
 [0.7091618  0.29083818]
 [0.02615333 0.9738467 ]
 [0.00390053 0.9960995 ]
